In [ ]:
# create the deck of tiles
def create_deck():    
    # wind tiles are formmated as ('Direction', 'x')
    # flower tiles are formmated as ('Flower', 'x')
    # joker tiles are formmated as ('Joker', 'x')
    # numbered tiles are formatted as (value, 'Suit')
        # note: dragons are formated as a numbered tile with value = 0
    bam = []
    crack = []
    dot = []
    for i in range(10):
        bam += [(i,'Bam')]
        crack += [(i,'Crack')]
        dot += [(i,'Dot')]
    numbered = bam*4+crack*4+dot*4
    wind = [('North','x')]*4+[('East','x')]*4+[('West','x')]*4+[('South','x')]*4
    flower = [('Flower','x')]*8
    joker = [('Joker','x')]*8
    deck = numbered + wind + flower + joker
    
    return deck

In [ ]:
# draw randomly from the deck of tiles (helpful for development)
def draw_hand(deck):
    taken = []
    hand = []
    while len(hand)<13:
        i = random.randint(0, 151)
        if i not in taken:
            taken += [i]
            tile = deck[i]
            hand += [tile]
    return hand

In [ ]:
# get count of needed tiles
def final_needed(card_hand, your_hand):
    if Counter(card_hand) != Counter({('Flower', 'x'): 2,(2, 'Dot'): 2,(0, 'Dot'): 4,(2, 'Crack'): 2,('North', 'x'): 1,('East', 'x'): 1,('West', 'x'): 1,('South', 'x'): 1}):
        jokerable = {}
        for key in Counter(card_hand):
            if Counter(card_hand)[key] > 2:
                 jokerable[key] = Counter(card_hand)[key]         
        needed = Counter(card_hand)-(Counter(card_hand) & Counter(your_hand))
        jokerable_needed = dict(needed & Counter(jokerable))
        non_jokerable_needed = dict(needed - Counter(jokerable))
    else:
        jokerable_needed = {}
        needed = Counter(card_hand)-(Counter(card_hand) & Counter(your_hand))
        non_jokerable_needed = dict(needed)
    
    jokers = Counter(your_hand)[('Joker','x')]
    return max((sum(jokerable_needed.values())-jokers),0)+(sum(needed.values())-sum(jokerable_needed.values()))

In [ ]:
# get count of jokerable needed tiles (tiles in groups of more than 2 of the same tile)
def count_jokerable_needed(card_hand, your_hand): 
    if Counter(card_hand) != Counter({('Flower', 'x'): 2,(2, 'Dot'): 2,(0, 'Dot'): 4,(2, 'Crack'): 2,('North', 'x'): 1,('East', 'x'): 1,('West', 'x'): 1,('South', 'x'): 1}):
        jokerable = {}
        for key in Counter(card_hand):
            if Counter(card_hand)[key] > 2:
                 jokerable[key] = Counter(card_hand)[key]         
        needed = Counter(card_hand)-(Counter(card_hand) & Counter(your_hand))
        jokerable_needed = dict(needed & Counter(jokerable))
    else:
        jokerable_needed = {}
        
    return sum(jokerable_needed.values())

In [ ]:
# get probabilities of getting all the needed tiles
def prob(card_hand, visibility, your_hand, alive):
   
    # get jokerable_needed and non_jokerable_needed
    if Counter(card_hand) != Counter({('Flower', 'x'): 2,(2, 'Dot'): 2,(0, 'Dot'): 4,(2, 'Crack'): 2,('North', 'x'): 1,('East', 'x'): 1,('West', 'x'): 1,('South', 'x'): 1}):
        jokerable = {}
        for key in Counter(card_hand):
            if Counter(card_hand)[key] > 2:
                 jokerable[key] = Counter(card_hand)[key]         
        needed = Counter(card_hand)-(Counter(card_hand) & Counter(your_hand))
        jokerable_needed = dict(needed & Counter(jokerable))
        non_jokerable_needed = dict(needed - Counter(jokerable))
    else:
        jokerable_needed = {}
        needed = Counter(card_hand)-(Counter(card_hand) & Counter(your_hand))
        non_jokerable_needed = dict(needed)

    # non_jokerable_needed probabilities
    non_jokerable_prob = []
    for key in non_jokerable_needed:
        for i in range(non_jokerable_needed[key]):
            non_jokerable_prob += [alive[key]-i]
    
    # jokerable_needed probabilities
    jokers = Counter(your_hand)[('Joker','x')]
    alive_jokers = alive[('Joker','x')]
    jokerable_prob_1 = []
    jokerable_prob = []
    for key in jokerable_needed:
        for i in range(jokerable_needed[key]):
            jokerable_prob_1 += [alive[key]-i]
    if len(jokerable_prob_1)>0:
        jokerable_prob_1 = [np.prod(jokerable_prob_1)**(1/len(jokerable_prob_1))]*(len(jokerable_prob_1)-jokers)
        # add probability of picking a joker
        for i in range(len(jokerable_prob_1)):
            jokerable_prob += [jokerable_prob_1[i]+((alive_jokers-i)/4)]
    else:
        jokerable_prob = jokerable_prob_1

    # combine and divide by size of alive (tiles left to pick from) 
    prob = non_jokerable_prob + jokerable_prob
    numerator = 1
    for i in prob:
        numerator *=i

    alive_size = sum(alive.values())
    denominator = 1
    for i in range(len(prob)):
        denominator *= alive_size
        alive_size -= 1
        
    # correct for visibility
    if visibility=='Concealed':
        denominator*=4

    return numerator/denominator

In [6]:
# indicates the tiles to call for
def call_for(card_hand, visibility, your_hand):    
    if visibility=='Concealed':
        return []
    elif Counter(card_hand) == Counter({('Flower', 'x'): 2,(2, 'Dot'): 2,(0, 'Dot'): 4,(2, 'Crack'): 2,('North', 'x'): 1,('East', 'x'): 1,('West', 'x'): 1,('South', 'x'): 1}):
        return []
    else:
        call = []
        jokers = Counter(your_hand)[('Joker','x')]
        jokerable = {}
        for key in Counter(card_hand):
            if Counter(card_hand)[key] > 2:
                 jokerable[key] = Counter(card_hand)[key]         
        needed = Counter(card_hand)-(Counter(card_hand) & Counter(your_hand))
        jokerable_needed = dict(needed & Counter(jokerable))
        for key in jokerable_needed:
            if jokerable_needed[key]-jokers<=1:
                call += [key]
        return call

In [7]:
# apply functions to card hands
def run_hand(df_full,your_hand, alive):
    df_full['Count Needed'] =  df_full['Hand'].apply(final_needed, your_hand = your_hand)
    df_full['Count Jokerable Needed'] =  df_full['Hand'].apply(count_jokerable_needed, your_hand = your_hand)
    df_full['Call For'] = df_full.apply(lambda x: call_for(x.Hand, x.Visibility, your_hand = your_hand), axis=1)
    df_full['Probability']= df_full.apply(lambda x: prob(x.Hand, x.Visibility, your_hand = your_hand, alive= alive), axis=1)
    
    df_full.sort_values('Probability',ascending=False, inplace=True)
    df_full.reset_index(drop=True, inplace=True)
    
    return df_full

In [ ]:
# figure out if a certain tile exists in each of the considered hands
def contains(hand, tile):
    if tile in hand:
        return 1
    else:
        return 0

In [ ]:
# function to count how many times a certain tile appears in each hand
def tile_count(hand, tile):
    if tile in hand:
        out = hand[tile]
        return out
    else:
        return 0

In [1]:
# get the value of each tile depending on your_hand and top portion of card hands considered
def tile_values(df_full, your_hand, stop_value):
    
    # stop_value
    considered = df_full.iloc[:stop_value,:]
    
    # split your_hand into singles and multiples
    jokers = 0
    singles = []
    multiples = {}
    for i in Counter(your_hand):
        if i == ('Joker', 'x'):
            jokers = Counter(your_hand)[i]
        elif Counter(your_hand)[i]==1:
            singles += [i]
        else:
            multiples[i] = Counter(your_hand)[i]
            
    # singles
    tiles = pd.DataFrame(columns = ["Tile","Probability"])
    for i in singles:
        df = pd.concat([considered['Probability'],considered['Hand'].apply(contains, tile = i)], axis=1)
        exist = df[df['Hand']==1]
        prob = sum(exist['Probability'])
        tiles = tiles.append({'Tile':i,"Probability":prob},ignore_index=True)
        
    # multiples
    for i in multiples:
        hand_contain = considered['Hand'].apply(contains, tile = i)
        hand_count = considered['Hand'].apply(tile_count, tile = i)
        df = pd.concat([considered['Probability'],hand_count.rename('Count'),hand_contain.rename('Contain')],axis=1)

        # get multiples
        count_running = 0
        prob_running = 0
        for j in reversed(range(1,multiples[i]+1)):
            tile = str(i)+' x '+str(j)
            c = df[df['Count']>=j]
            prob = sum(c['Probability'])
            tiles = tiles.append({'Tile':tile,"Probability":prob},ignore_index=True)
            
    # jokers
    df = pd.concat([considered['Probability'],considered['Count Jokerable Needed']],axis=1)
    for i in range(jokers):
        tile = "('Joker', 'x') x "+str(i+1)
        prob = sum(df[df['Count Jokerable Needed']-i>0]['Probability'])
        tiles = tiles.append({'Tile':tile,"Probability":prob},ignore_index=True)

    tiles.sort_values('Probability', ascending=False, inplace=True)
    tiles.reset_index(drop=True,inplace=True)
    
    return tiles

In [1]:
# get the optimal tiles to discard considering at least the most 5 probable hands (but up to all possible hands)
def optimize_values(df_full, your_hand):
    if len(your_hand)>12:
        i = 5
        tiles = tile_values(df_full, your_hand, i)
        while (tiles.iloc[-3,1]==tiles.iloc[-4,1])and(i<644):
            i += 1
            tiles = tile_values(df_full, your_hand, i)
        return i, tiles
    else:
        return 'NA', 'NA'

In [8]:
# discard dead tiles
def thrown_tiles(alive,tile):
    alive = alive - Counter(tile)
    return alive

In [6]:
# add tiles to your_hand and run analysis for trading
# bams
def bam0():
    global your_hand, alive
    your_hand += [(0,'Bam')]   
    alive -= Counter([(0,'Bam')])
    update()
def bam1():
    global your_hand, alive
    your_hand += [(1,'Bam')]
    alive -= Counter([(1,'Bam')])
    update()
def bam2():
    global your_hand, alive
    your_hand += [(2,'Bam')] 
    alive -= Counter([(2,'Bam')])
    update()
def bam3():
    global your_hand, alive
    your_hand += [(3,'Bam')]
    alive -= Counter([(3,'Bam')])
    update()
def bam4():
    global your_hand, alive
    your_hand += [(4,'Bam')]   
    alive -= Counter([(4,'Bam')])
    update()
def bam5():
    global your_hand, alive
    your_hand += [(5,'Bam')]
    alive -= Counter([(5,'Bam')])
    update()
def bam6():
    global your_hand, alive
    your_hand += [(6,'Bam')]
    alive -= Counter([(6,'Bam')])
    update()
def bam7():
    global your_hand, alive
    your_hand += [(7,'Bam')]
    alive -= Counter([(7,'Bam')])
    update()
def bam8():
    global your_hand, alive
    your_hand += [(8,'Bam')]
    alive -= Counter([(8,'Bam')])
    update()
def bam9():
    global your_hand, alive
    your_hand += [(9,'Bam')]
    alive -= Counter([(9,'Bam')])
    update()
# cracks
def crack0():
    global your_hand, alive
    your_hand += [(0,'Crack')] 
    alive -= Counter([(0,'Crack')])
    update()
def crack1():
    global your_hand, alive
    your_hand += [(1,'Crack')]
    alive -= Counter([(1,'Crack')])
    update()
def crack2():
    global your_hand, alive
    your_hand += [(2,'Crack')] 
    alive -= Counter([(2,'Crack')])
    update()
def crack3():
    global your_hand, alive
    your_hand += [(3,'Crack')]
    alive -= Counter([(3,'Crack')])
    update()
def crack4():
    global your_hand, alive
    your_hand += [(4,'Crack')]
    alive -= Counter([(4,'Crack')])
    update()
def crack5():
    global your_hand, alive
    your_hand += [(5,'Crack')]
    alive -= Counter([(5,'Crack')])
    update()
def crack6():
    global your_hand, alive
    your_hand += [(6,'Crack')]
    alive -= Counter([(6,'Crack')])
    update()
def crack7():
    global your_hand, alive
    your_hand += [(7,'Crack')]
    alive -= Counter([(7,'Crack')])
    update()
def crack8():
    global your_hand, alive
    your_hand += [(8,'Crack')]
    alive -= Counter([(8,'Crack')])
    update()
def crack9():
    global your_hand, alive
    your_hand += [(9,'Crack')]
    alive -= Counter([(9,'Crack')])
    update()
# dots
def dot0():
    global your_hand, alive
    your_hand += [(0,'Dot')]  
    alive -= Counter([(0,'Dot')])
    update()
def dot1():
    global your_hand, alive
    your_hand += [(1,'Dot')]
    alive -= Counter([(1,'Dot')])
    update()
def dot2():
    global your_hand, alive
    your_hand += [(2,'Dot')] 
    alive -= Counter([(2,'Dot')])
    update()
def dot3():
    global your_hand, alive
    your_hand += [(3,'Dot')]
    alive -= Counter([(3,'Dot')])
    update()
def dot4():
    global your_hand, alive
    your_hand += [(4,'Dot')] 
    alive -= Counter([(4,'Dot')])
    update()
def dot5():
    global your_hand, alive
    your_hand += [(5,'Dot')]
    alive -= Counter([(5,'Dot')])
    update()
def dot6():
    global your_hand, alive
    your_hand += [(6,'Dot')] 
    alive -= Counter([(6,'Dot')])
    update()
def dot7():
    global your_hand, alive
    your_hand += [(7,'Dot')]
    alive -= Counter([(7,'Dot')])
    update()
def dot8():
    global your_hand, alive
    your_hand += [(8,'Dot')] 
    alive -= Counter([(8,'Dot')])
    update()
def dot9():
    global your_hand, alive
    your_hand += [(9,'Dot')]
    alive -= Counter([(9,'Dot')])
    update()
# winds
def north():
    global your_hand, alive
    your_hand += [('North','x')]  
    alive -= Counter([('North','x')])
    update()
def east():
    global your_hand, alive
    your_hand += [('East','x')]
    alive -= Counter([('East','x')])
    update()
def west():
    global your_hand, alive
    your_hand += [('West','x')]   
    alive -= Counter([('West','x')])
    update()
def south():
    global your_hand, alive
    your_hand += [('South','x')]
    alive -= Counter([('South','x')])
    update()
# jokers and flowers
def joker():
    global your_hand, alive
    your_hand += [('Joker','x')]   
    alive -= Counter([('Joker','x')])
    update()
def flower():
    global your_hand, alive
    your_hand += [('Flower','x')]
    alive -= Counter([('Flower','x')])
    update()
def reset_deck():
    global alive
    alive = Counter(create_deck())
    update()
def clear_hand():
    global your_hand
    your_hand = []
    update()
def update():
    global your_hand, df_full, alive
    tile_count_2["text"]=str(len(your_hand))
    your_hand_displayed["text"]= str(dict(Counter(your_hand)))
    hands_considered["text"]= str(optimize_values(run_hand(df_full, your_hand, alive), your_hand)[0])
    tile_values_2["text"]= str(optimize_values(run_hand(df_full, your_hand, alive), your_hand)[1])
    top_3_hands_1["text"]= str(df_full.iloc[0:3,2])
    top_3_hands_2["text"]= str(df_full.iloc[0:3,[4,3,5,6,9,8]])
    alive_df = pd.DataFrame({'TILE':list(alive.keys()),'REMAINING':list(alive.values())})
    alive_keys["text"]=str(alive_df.iloc[:,0])
    alive_values["text"]=str(alive_df.iloc[:,1])
    with open('alive_minus_hand.txt','w+') as f:
        f.write(str(alive-Counter(your_hand)))

In [13]:
# remove tiles from LIVE DECK for game play
# bams
def b0():
    global alive  
    alive -= Counter([(0,'Bam')])
    update()
def b1():
    global alive
    alive -= Counter([(1,'Bam')])
    update()
def b2():
    global alive
    alive -= Counter([(2,'Bam')])
    update()
def b3():
    global alive
    alive -= Counter([(3,'Bam')])
    update()
def b4():
    global alive
    alive -= Counter([(4,'Bam')])
    update()
def b5():
    global alive
    alive -= Counter([(5,'Bam')])
    update()
def b6():
    global alive
    alive -= Counter([(6,'Bam')])
    update()
def b7():
    global alive
    alive -= Counter([(7,'Bam')])
    update()
def b8():
    global alive
    alive -= Counter([(8,'Bam')])
    update()
def b9():
    global alive
    alive -= Counter([(9,'Bam')])
    update()
# cracks
def c0():
    global alive 
    alive -= Counter([(0,'Crack')])
    update()
def c1():
    global alive
    alive -= Counter([(1,'Crack')])
    update()
def c2():
    global alive
    alive -= Counter([(2,'Crack')])
    update()
def c3():
    global alive
    alive -= Counter([(3,'Crack')])
    update()
def c4():
    global alive
    alive -= Counter([(4,'Crack')])
    update()
def c5():
    global alive
    alive -= Counter([(5,'Crack')])
    update()
def c6():
    global alive
    alive -= Counter([(6,'Crack')])
    update()
def c7():
    global alive
    alive -= Counter([(7,'Crack')])
    update()
def c8():
    global alive
    alive -= Counter([(8,'Crack')])
    update()
def c9():
    global alive
    alive -= Counter([(9,'Crack')])
    update()
# dots
def d0():
    global alive
    alive -= Counter([(0,'Dot')])
    update()
def d1():
    global alive
    alive -= Counter([(1,'Dot')])
    update()
def d2():
    global alive
    alive -= Counter([(2,'Dot')])
    update()
def d3():
    global alive
    alive -= Counter([(3,'Dot')])
    update()
def d4():
    global alive
    alive -= Counter([(4,'Dot')])
    update()
def d5():
    global alive
    alive -= Counter([(5,'Dot')])
    update()
def d6():
    global alive
    alive -= Counter([(6,'Dot')])
    update()
def d7():
    global alive
    alive -= Counter([(7,'Dot')])
    update()
def d8():
    global alive
    alive -= Counter([(8,'Dot')])
    update()
def d9():
    global alive
    alive -= Counter([(9,'Dot')])
    update()
# winds
def n():
    global alive
    alive -= Counter([('North','x')])
    update()
def e():
    global alive
    alive -= Counter([('East','x')])
    update()
def w():
    global alive  
    alive -= Counter([('West','x')])
    update()
def s():
    global alive
    alive -= Counter([('South','x')])
    update()
# jokers and flowers
def j():
    global alive 
    alive -= Counter([('Joker','x')])
    update()
def f():
    global alive
    alive -= Counter([('Flower','x')])
    update()

In [ ]:
# remove tiles from YOUR HAND for game play
# bams
def ba0():
    global your_hand 
    x = (0,'Bam')
    if x in your_hand: your_hand.remove(x)
    update()
def ba1():
    global your_hand
    x = (1,'Bam')
    if x in your_hand: your_hand.remove(x)
    update()
def ba2():
    global your_hand
    x = (2,'Bam')
    if x in your_hand: your_hand.remove(x)
    update()
def ba3():
    global your_hand
    x = (3,'Bam')
    if x in your_hand: your_hand.remove(x)
    update()
def ba4():
    global your_hand
    x = (4,'Bam')
    if x in your_hand: your_hand.remove(x)
    update()
def ba5():
    global your_hand
    x = (5,'Bam')
    if x in your_hand: your_hand.remove(x)
    update()
def ba6():
    global your_hand
    x = (6,'Bam')
    if x in your_hand: your_hand.remove(x)
    update()
def ba7():
    global your_hand
    x = (7,'Bam')
    if x in your_hand: your_hand.remove(x)
    update()
def ba8():
    global your_hand
    x = (8,'Bam')
    if x in your_hand: your_hand.remove(x)
    update()
def ba9():
    global your_hand
    x = (9,'Bam')
    if x in your_hand: your_hand.remove(x)
    update()
# cracks
def cr0():
    global your_hand 
    x = (0,'Crack')
    if x in your_hand: your_hand.remove(x)
    update()
def cr1():
    global your_hand
    x = (1,'Crack')
    if x in your_hand: your_hand.remove(x)
    update()
def cr2():
    global your_hand
    x = (2,'Crack')
    if x in your_hand: your_hand.remove(x)
    update()
def cr3():
    global your_hand
    x = (3,'Crack')
    if x in your_hand: your_hand.remove(x)
    update()
def cr4():
    global your_hand
    x = (4,'Crack')
    if x in your_hand: your_hand.remove(x)
    update()
def cr5():
    global your_hand
    x = (5,'Crack')
    if x in your_hand: your_hand.remove(x)
    update()
def cr6():
    global your_hand
    x = (6,'Crack')
    if x in your_hand: your_hand.remove(x)
    update()
def cr7():
    global your_hand
    x = (7,'Crack')
    if x in your_hand: your_hand.remove(x)
    update()
def cr8():
    global your_hand
    x = (8,'Crack')
    if x in your_hand: your_hand.remove(x)
    update()
def cr9():
    global your_hand
    x = (9,'Crack')
    if x in your_hand: your_hand.remove(x)
    update()
# dots
def do0():
    global your_hand
    x = (0,'Dot')
    if x in your_hand: your_hand.remove(x)
    update()
def do1():
    global your_hand
    x = (1,'Dot')
    if x in your_hand: your_hand.remove(x)
    update()
def do2():
    global your_hand
    x = (2,'Dot')
    if x in your_hand: your_hand.remove(x)
    update()
def do3():
    global your_hand
    x = (3,'Dot')
    if x in your_hand: your_hand.remove(x)
    update()
def do4():
    global your_hand
    x = (4,'Dot')
    if x in your_hand: your_hand.remove(x)
    update()
def do5():
    global your_hand
    x = (5,'Dot')
    if x in your_hand: your_hand.remove(x)
    update()
def do6():
    global your_hand
    x = (6,'Dot')
    if x in your_hand: your_hand.remove(x)
    update()
def do7():
    global your_hand
    x = (7,'Dot')
    if x in your_hand: your_hand.remove(x)
    update()
def do8():
    global your_hand
    x = (8,'Dot')
    if x in your_hand: your_hand.remove(x)
    update()
def do9():
    global your_hand
    x = (9,'Dot')
    if x in your_hand: your_hand.remove(x)
    update()
# winds
def no():
    global your_hand
    x = ('North','x')
    if x in your_hand: your_hand.remove(x)
    update()
def ea():
    global your_hand
    x = ('East','x')
    if x in your_hand: your_hand.remove(x)
    update()
def we():
    global your_hand  
    x = ('West','x')
    if x in your_hand: your_hand.remove(x)
    update()
def so():
    global your_hand
    x = ('South','x')
    if x in your_hand: your_hand.remove(x)
    update()
# jokers and flowers
def jo():
    global your_hand 
    x = ('Joker','x')
    if x in your_hand: your_hand.remove(x)
    update()
def fl():
    global your_hand
    x = ('Flower','x')
    if x in your_hand: your_hand.remove(x)
    update()